In [16]:
import qiskit as qk
import numpy as np
from copy import deepcopy
from qiskit.circuit.random import random_circuit

In [23]:
class ansatz:
    def __init__(self,n_qubits,reps):
        self.n_params = 2*reps*n_qubits
        self.n_qubits = n_qubits
        self.reps = reps
    
    def __call__(self,circuit,theta,inverse=False):
        i = 0
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
        for rep in range(self.reps):
            for qubit in range(self.n_qubits):
                circ.ry(theta[i],q_reg[qubit])
                i+=1
                circ.rz(theta[i],q_reg[qubit])
                i+= 1
            for qubit in range(self.n_qubits-1):
                circ.cx(q_reg[qubit],q_reg[qubit+1])
                
        if inverse:
            circ = circ.inverse()
        return(circuit+circ)
    
class ansatz2:
    def __init__(self,n_qubits,reps):
        self.n_params = 2*reps*n_qubits
        self.n_qubits = n_qubits
        self.reps = reps
    
    def __call__(self,circuit,theta,inverse=False):
        i = 0
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
        for rep in range(self.reps):
            
            if rep%2 == 0:
                for qubit in range(self.n_qubits-1):
                    circ.cx(q_reg[qubit],q_reg[qubit+1])
                    
            else:
                for qubit in reversed(range(self.n_qubits-1)):
                    circ.cx(q_reg[qubit],q_reg[qubit+1])
            
            
            for qubit in range(self.n_qubits):
                circ.ry(theta[i],q_reg[qubit])
                i += 1
                circ.rz(theta[i],q_reg[qubit])
                i += 1
                       
                
        if inverse:
            circ = circ.inverse()
        return(circuit+circ)
    
class ansatz3:
    def __init__(self,n_qubits,reps):
        self.n_params = 2*reps*n_qubits
        self.n_qubits = n_qubits
        self.reps = reps
    
    def __call__(self,circuit,theta,inverse=False):
        i = 0
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
        for rep in range(self.reps):
            for qubit in range(self.n_qubits):
                circ.ry(theta[i],q_reg[qubit])
                i+=1
                
            for qubit in range(0, self.n_qubits, 2):
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                circ.rz(theta[i],q_reg[qubit+1])
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                i+=1
                
            
                
            circ.cx(q_reg[0], q_reg[-1])
            circ.rz(theta[i], q_reg[-1])
            circ.cx(q_reg[0], q_reg[-1])
            i+=1
    
        if inverse:
            circ = circ.inverse()
        return(circuit+circ)
    
class Adam():
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = None
        self.v = None
        self.t = None

    def initialize(self, dims):
        self.m = []
        self.v = []
        self.t = 0

        for dim in dims:
            self.m.append(np.zeros(dim))
            self.v.append(np.zeros(dim))

    def __call__(self, weight_gradient_list):
        self.t += 1
        weight_gradient_modified = []

        for grad, m_, v_ in zip(weight_gradient_list, self.m, self.v):
            m_[:] = self.beta1 * m_ + (1 - self.beta1) * grad
            v_[:] = self.beta2 * v_ + (1 - self.beta2) * grad**2

            m_hat = m_ / (1 - self.beta1**self.t)
            v_hat = v_ / (1 - self.beta2**self.t)
            grad_modified = m_hat / (np.sqrt(v_hat) + self.eps)
            weight_gradient_modified.append(grad_modified)

        return weight_gradient_modified

In [17]:
np.random.seed(42)

n_qubits = 4
depth = 20
circuit = random_circuit(n_qubits,depth)

print(len(circuit))

In [24]:
np.random.seed(42)

class RCO:
    def __init__(self,ansatz,optimizer=None,shots=10000):
        self.n_params = ansatz.n_params
        self.ansatz = ansatz
        self.shots=shots
        self.optimizer=optimizer

    def divide_circuit(self,circuit,divisor):
        circuit_size = len(circuit)
        gates_per_sub_circuit = int(circuit_size/divisor)
        k = 0
        circuit_list = []
        while k < circuit_size:
            circ = deepcopy(circuit)
            for i in range(k):
                circ.data.pop(0)
            for i in range(circuit_size - gates_per_sub_circuit - k):
                circ.data.pop(-1)
            circuit_list.append(deepcopy(circ))
            k+= gates_per_sub_circuit
        return(circuit_list)
    
    def __call__(self,circuit,theta,theta_prev=None):
        
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
            
            
        circ = self.ansatz(circ,theta)
        circ += circuit.inverse()
        if not theta_prev is None:
            circ = self.ansatz(circ,theta_prev,inverse=True)
        c_reg = qk.ClassicalRegister(q_reg.size)
        circ.add_register(c_reg)
        circ.measure(q_reg,c_reg)
        shots=self.shots
        
        job = qk.execute(circ,
                        backend=qk.Aer.get_backend('qasm_simulator'),
                        shots=shots,
                        )
        results = job.result()
        results = results.get_counts(circ)

        probs = np.zeros(q_reg.size)
        for key,value in results.items():
            key_ = key[::-1]
            for i,bit in enumerate(key_):
                if bit == '1':
                    probs[i] += value
        probs /= shots
        probs = np.mean(probs)
        return(probs)
    
    def get_gradients(self,circuit,theta,theta_prev=None):
        grads = np.zeros(theta.shape)
        for i,param in enumerate(theta):
            theta[i] += np.pi/2
            probs_plus = self(circuit,theta,theta_prev)
            theta[i] -= np.pi
            probs_minus = self(circuit,theta,theta_prev)
            theta[i] += np.pi/2
            grads[i] += (probs_plus - probs_minus)/2
        return(grads)
    
    def gradient_descent(self,circuit,theta,theta_prev=None,lr=0.1,max_iters=1000,tol = 1e-3):
        self.optimizer.initialize([self.ansatz.n_params])
        for epoch in range(max_iters):
            grad = self.get_gradients(circuit,theta,theta_prev)
            grad = self.optimizer([grad])[0]
            theta  = theta - lr*grad
            probs = self(circuit,theta,theta_prev)
            print(epoch,probs)
            if probs <= tol:
                break
        return(theta)
            
    
    def optimize(self,circuit,divisor,lr=0.1,max_iters=1000,tol=1e-3):
        circuit_list = self.divide_circuit(circuit,divisor)
        thetas = np.random.uniform(-np.pi, np.pi, (len(circuit_list), self.n_params))
        for i, circuit in enumerate(circuit_list):
            print(i)
            print(circuit)
            if i != 0:
                thetas[i,:] = np.copy(thetas[i-1,:])
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],thetas[i-1,:],lr=lr,max_iters=max_iters,tol=tol)
                
            else:
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],lr=lr,max_iters=max_iters,tol=tol)
                
        return(thetas[-1,:])
                   
reps = 4 
circuit_optimizer = RCO(ansatz2(n_qubits,reps),optimizer=Adam(), shots=10000)
theta = circuit_optimizer.optimize(circuit,5,tol=1e-3,lr=0.05)

0
     ┌───┐┌───────────────────────────┐┌───┐                        
q_0: ┤ X ├┤ U3(4.2136,0.76276,5.1861) ├┤ H ├────────────────────────
     └─┬─┘└──────────┬─────┬──────────┘├───┤                        
q_1: ──┼─────────────┤ TDG ├───────────┤ X ├──■────■────────────────
       │             └┬───┬┘           └───┘┌─┴─┐  │  ┌────────────┐
q_2: ──■──────────────┤ T ├─────────────────┤ X ├──┼──┤ RX(5.2685) ├
       │             ┌┴───┴┐                └─┬─┘┌─┴─┐└────────────┘
q_3: ──■─────────────┤ SDG ├──────────────────■──┤ Y ├──────────────
                     └─────┘                     └───┘              
0 0.520575
1 0.46907499999999996
2 0.427975
3 0.38815
4 0.35609999999999997
5 0.331825
6 0.30552500000000005
7 0.27925
8 0.2551
9 0.2284
10 0.191875
11 0.1658
12 0.1369
13 0.1124
14 0.09412499999999999
15 0.07924999999999999
16 0.070525
17 0.059899999999999995
18 0.051074999999999995
19 0.039650000000000005
20 0.032175
21 0.0239
22 0.022824999999999998
23 0.02124999999999999

39 0.022075
40 0.0204
41 0.0196
42 0.01845
43 0.0189
44 0.017
45 0.0177
46 0.016425000000000002
47 0.016275
48 0.016375
49 0.015025
50 0.0144
51 0.014425
52 0.012225
53 0.011975
54 0.013175
55 0.011700000000000002
56 0.012199999999999999
57 0.011049999999999999
58 0.010324999999999999
59 0.009875
60 0.009875
61 0.01
62 0.009474999999999999
63 0.010125
64 0.008925
65 0.008475
66 0.009174999999999999
67 0.009475
68 0.008275000000000001
69 0.007549999999999999
70 0.007875
71 0.007175
72 0.006775
73 0.00705
74 0.007075
75 0.0075250000000000004
76 0.006150000000000001
77 0.005700000000000001
78 0.005475
79 0.005775
80 0.0055000000000000005
81 0.006125
82 0.005475000000000001
83 0.005025
84 0.006025
85 0.004999999999999999
86 0.005
87 0.005599999999999999
88 0.004175
89 0.0043
90 0.004675
91 0.0046
92 0.003725
93 0.0037749999999999997
94 0.0047
95 0.003825
96 0.0030499999999999998
97 0.003725
98 0.0039
99 0.0038250000000000003
100 0.003
101 0.0039250000000000005
102 0.00435
103 0.003075
104 

In [29]:
ans = ansatz2(n_qubits,reps)
circ = ans(circuit,theta,inverse=True)

q_reg = circ.qregs[0]
c_reg = qk.ClassicalRegister(q_reg.size)
circ.add_register(c_reg)
circ.measure(q_reg,c_reg)
shots=10000

job = qk.execute(circ,
                backend=qk.Aer.get_backend('qasm_simulator'),
                shots=shots,
                )
results = job.result()
results = results.get_counts(circ)

probs = np.zeros(q_reg.size)
for key,value in results.items():
    print(key,value)
    key_ = key[::-1]
    for i,bit in enumerate(key_):
        if bit == '1':
            probs[i] += value
probs /= shots
print(probs)

1000 3
0110 1
0100 3
0010 5
0101 4
1100 3
0001 3
1110 6
0111 1
0011 8
0000 9866
1101 21
1011 13
1010 5
1111 28
1001 30
[0.0108 0.0067 0.0067 0.0109]


In [32]:
np.random.seed(42)

n_qubits = 5
depth = 20
circuit = random_circuit(n_qubits,depth)

print(len(circuit))

59


In [33]:
np.random.seed(42)

class RCO:
    def __init__(self,ansatz,optimizer=None,shots=10000):
        self.n_params = ansatz.n_params
        self.ansatz = ansatz
        self.shots=shots
        self.optimizer=optimizer

    def divide_circuit(self,circuit,divisor):
        circuit_size = len(circuit)
        gates_per_sub_circuit = int(circuit_size/divisor)
        k = 0
        circuit_list = []
        while k < circuit_size:
            circ = deepcopy(circuit)
            for i in range(k):
                circ.data.pop(0)
            for i in range(circuit_size - gates_per_sub_circuit - k):
                circ.data.pop(-1)
            circuit_list.append(deepcopy(circ))
            k+= gates_per_sub_circuit
        return(circuit_list)
    
    def __call__(self,circuit,theta,theta_prev=None):
        
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
            
            
        circ = self.ansatz(circ,theta)
        circ += circuit.inverse()
        if not theta_prev is None:
            circ = self.ansatz(circ,theta_prev,inverse=True)
        c_reg = qk.ClassicalRegister(q_reg.size)
        circ.add_register(c_reg)
        circ.measure(q_reg,c_reg)
        shots=self.shots
        
        job = qk.execute(circ,
                        backend=qk.Aer.get_backend('qasm_simulator'),
                        shots=shots,
                        )
        results = job.result()
        results = results.get_counts(circ)

        probs = np.zeros(q_reg.size)
        for key,value in results.items():
            key_ = key[::-1]
            for i,bit in enumerate(key_):
                if bit == '1':
                    probs[i] += value
        probs /= shots
        probs = np.mean(probs)
        return(probs)
    
    def get_gradients(self,circuit,theta,theta_prev=None):
        grads = np.zeros(theta.shape)
        for i,param in enumerate(theta):
            theta[i] += np.pi/2
            probs_plus = self(circuit,theta,theta_prev)
            theta[i] -= np.pi
            probs_minus = self(circuit,theta,theta_prev)
            theta[i] += np.pi/2
            grads[i] += (probs_plus - probs_minus)/2
        return(grads)
    
    def gradient_descent(self,circuit,theta,theta_prev=None,lr=0.1,max_iters=1000,tol = 1e-3):
        self.optimizer.initialize([self.ansatz.n_params])
        for epoch in range(max_iters):
            grad = self.get_gradients(circuit,theta,theta_prev)
            grad = self.optimizer([grad])[0]
            theta  = theta - lr*grad
            probs = self(circuit,theta,theta_prev)
            print(epoch,probs)
            if probs <= tol:
                break
        return(theta)
            
    
    def optimize(self,circuit,divisor,lr=0.1,max_iters=1000,tol=1e-3):
        circuit_list = self.divide_circuit(circuit,divisor)
        thetas = np.random.uniform(-np.pi, np.pi, (len(circuit_list), self.n_params))
        for i, circuit in enumerate(circuit_list):
            print(i)
            print(circuit)
            if i != 0:
                thetas[i,:] = np.copy(thetas[i-1,:])
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],thetas[i-1,:],lr=lr,max_iters=max_iters,tol=tol)
                
            else:
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],lr=lr,max_iters=max_iters,tol=tol)
                
        return(thetas[-1,:])
                   
reps = 4 
circuit_optimizer = RCO(ansatz2(n_qubits,reps),optimizer=Adam(), shots=10000)
theta = circuit_optimizer.optimize(circuit,5,tol=1e-3,lr=0.05)

0
                 ┌───┐            ┌───┐┌───┐                        »
q_0: ────────────┤ T ├────────────┤ X ├┤ I ├───────────■────────────»
     ┌───────────┴───┴───────────┐└─┬─┘└───┘         ┌─┴─┐          »
q_1: ┤ U3(1.6816,4.2136,0.76276) ├──┼────■────■──────┤ X ├──────────»
     └───────────────────────────┘  │    │    │      └───┘     ┌───┐»
q_2: ──────────────X────────────────■────┼────┼────────■───────┤ X ├»
                   │                │    │  ┌─┴─┐      │       └─┬─┘»
q_3: ──────────────X────────────────■────┼──┤ H ├──────┼─────────■──»
                   │                     │  └───┘┌─────┴──────┐  │  »
q_4: ──────────────■─────────────────────■───────┤ RZ(5.0519) ├──■──»
                                                 └────────────┘     »
«                                  
«q_0: ─────────────────────────────
«                                  
«q_1: ─────────────────────────────
«                                  
«q_2: ─────────────────────────────
«     ┌─────

KeyboardInterrupt: 